In [1]:
import pandas as pd
import numpy as np
import huggingface_hub
from huggingface_hub import hf_hub_download


In [2]:
!pip -q install langchain
!pip -q install langchain_community
!pip -q install langchain_experimental
!pip -q install duckduckgo_search
!pip -q install llama-cpp-python

In [3]:
from langchain.agents import Tool
import json
from langchain_experimental.utilities import PythonREPL
from langchain.tools import DuckDuckGoSearchRun,YahooFinanceNewsTool,GoogleSearchResults
from llama_cpp import Llama

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [4]:
!nvidia -smi

zsh:1: command not found: nvidia


In [5]:
model_name_or_path = "TheBloke/Mistral-7B-Instruct-v0.2-GGUF"
model_basename = "mistral-7b-instruct-v0.2.Q5_K_M.gguf"

In [6]:
model_path=hf_hub_download(repo_id=model_name_or_path,filename=model_basename)

In [7]:
llm=Llama(model_path=model_path,n_batch=512,n_ctx=6000,n_gpu_layers=43,n_threads=2)

llama_model_loader: loaded meta data with 24 key-value pairs and 291 tensors from /Users/saisampath/.cache/huggingface/hub/models--TheBloke--Mistral-7B-Instruct-v0.2-GGUF/snapshots/3a6fbf4a41a1d52e415a4958cde6856d34b2db93/mistral-7b-instruct-v0.2.Q5_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.2
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
lla

In [8]:
response=llm("what is the apple ")


llama_print_timings:        load time =    8075.29 ms
llama_print_timings:      sample time =       1.45 ms /    16 runs   (    0.09 ms per token, 11019.28 tokens per second)
llama_print_timings: prompt eval time =    8075.19 ms /     6 tokens ( 1345.87 ms per token,     0.74 tokens per second)
llama_print_timings:        eval time =     774.76 ms /    15 runs   (   51.65 ms per token,    19.36 tokens per second)
llama_print_timings:       total time =    8878.31 ms


In [9]:
response["choices"][0]["text"]

'2e memory architecture?\n\nThe Apple IIe, introduced in 1'

### Steps for the Rag Method

Step1: Upload the file and convert the the file in to tockenizer and then do embadding layer

Step2: store the the vectors into the Chrome DB

Step3:Create the Rag QandA

Step4:Check the response and evalute the response using the other LLM

In [10]:
# Step1
from langchain_community.document_loaders import PyPDFLoader,PyPDFDirectoryLoader
!pip install -q matplotlib-venn
!pip install -q pypdf
!pip install -q tiktoken
!pip install -q sentence-transformers
!pip install -q chromadb

In [11]:
data_path="HBR_How_Apple_Is_Organized_For_Innovation-4 (1).pdf"

In [12]:
data=PyPDFLoader(data_path)
# using the PyPdfloader we have loaded the Pdf file


In [13]:
#text spliter we have split the document using the REcursiveCharacterTextsplitting
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_spliter= RecursiveCharacterTextSplitter.from_tiktoken_encoder(encoding_name="cl100k_base",chunk_size=512,chunk_overlap=10)

In [14]:
#i9nthis step we are chuncking the data

apple_data_chuck=data.load_and_split(text_spliter)


### Vector Database store ChromeDB

In [15]:
Apple_Article= "HBR_How_Apple_Is_Organized_For_Innovation-4 (1).pdf"

In [16]:
# create the vector Db
from langchain_community.vectorstores import Chroma
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
embedding_model= SentenceTransformerEmbeddings(model_name="paraphrase-MiniLM-L6-v2")

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [17]:
vector_store= Chroma.from_documents(
    apple_data_chuck,
    embedding_model,
    collection_name="Apple_Article"
)

In [18]:
vector_store.search("apple value?",search_type="similarity")

[Document(metadata={'page': 2, 'source': 'HBR_How_Apple_Is_Organized_For_Innovation-4 (1).pdf'}, page_content='PHOTOGRAPHER\u2002MIKAEL JANSSONHow Apple Is  Organized  for InnovationIt’s about experts  leading experts.ORGANIZATIONAL \nCULTURE\nJoel M. \nPodolny\nDean, Apple \nUniversityMorten T. \nHansen\nFaculty, Apple \nUniversityAUTHORSFOR ARTICLE REPRINTS CALL 800-988-0886 OR 617-783-7500, OR VISIT HBR.ORG\nHarvard Business Review\nNovember–December 2020 \u20093\nThis article is made available to you with compliments of Apple Inc for your personal use. Further posting, copying or distribution is not permitted.'),
 Document(metadata={'page': 7, 'source': 'HBR_How_Apple_Is_Organized_For_Innovation-4 (1).pdf'}, page_content='8Harvard Business Review\nNovember–December 2020\nThis article is made available to you with compliments of Apple Inc for your personal use. Further posting, copying or distribution is not permitted.'),
 Document(metadata={'page': 1, 'source': 'HBR_How_Apple_Is_Or

In [19]:
retriver=vector_store.as_retriever(search_type="similarity",k=5,search_kwargs={'k':5})

### Creating the prompt Message

In [20]:
qna_system_message = """
You are an assistant whose work is to review the articles and provide the appropriate answers from the context.
User input will have the context required by you to answer user questions.
This context will begin with the token: ###Context.
The context contains references to specific portions of a document relevant to the user query.

User questions will begin with the token: ###Question.

Please answer only using the context provided in the input. Do not mention anything about the context in your final answer.

If the answer is not found in the context, respond "I don't know".
"""

In [21]:
qna_user_message_template = """
###Context
Here are some documents that are relevant to the question mentioned below.
{context}

###Question
{question}
"""

### Setting up the Rag System_message

In [22]:
def Rag(user_input):
  """We are taking the userinput
  and it will give the output for the vector Db
  and return the Relavent content """

  context= retriver.get_relevant_documents(user_input)
  context_list= [d.page_content for d in context]
  context_query=".".join(context_list)

  #using the prompt for the response
  prompt=f"""[INST]{qna_system_message}\n
  {"user"}:{qna_user_message_template.format(context=context_query,question=user_input)}"""
  try:
    response=llm(prompt=prompt,temperature=0,top_k=40,top_p=0.95,echo=False,repeat_penalty=1,stop=["INST"])
    prediction=response["choices"][0]["text"]
  except Exception as e:
    prediction=print(e)

  return prediction





In [25]:
response=Rag("Who are the authors of this article and who published this article ?")

Llama.generate: prefix-match hit

llama_print_timings:        load time =    8075.29 ms
llama_print_timings:      sample time =       2.72 ms /    16 runs   (    0.17 ms per token,  5886.68 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =    3549.38 ms /    16 runs   (  221.84 ms per token,     4.51 tokens per second)
llama_print_timings:       total time =    3588.21 ms


In [26]:
print(response)


Answer:
Joel M. Podolny and Morten T


In [28]:
print(Rag("Can you explain specific examples from the article where Apple's approach to leadership has led to successful innovations?"))

Llama.generate: prefix-match hit



I don't know. The article discusses Apple's approach to



llama_print_timings:        load time =    8075.29 ms
llama_print_timings:      sample time =       4.54 ms /    16 runs   (    0.28 ms per token,  3525.78 tokens per second)
llama_print_timings: prompt eval time =   16920.30 ms /  2056 tokens (    8.23 ms per token,   121.51 tokens per second)
llama_print_timings:        eval time =     949.65 ms /    15 runs   (   63.31 ms per token,    15.80 tokens per second)
llama_print_timings:       total time =   17984.71 ms


In [29]:
print(Rag("Why apple is a functional organization?"))

Llama.generate: prefix-match hit



###Answer
Apple is a functional organization because its leaders believe in



llama_print_timings:        load time =    8075.29 ms
llama_print_timings:      sample time =       3.05 ms /    16 runs   (    0.19 ms per token,  5244.18 tokens per second)
llama_print_timings: prompt eval time =   17719.62 ms /  2280 tokens (    7.77 ms per token,   128.67 tokens per second)
llama_print_timings:        eval time =     966.29 ms /    15 runs   (   64.42 ms per token,    15.52 tokens per second)
llama_print_timings:       total time =   18776.51 ms


In [30]:
print(Rag("List down the three leadership characteristics in bulleted points and explain each one of the characteristics under two lines."))

Llama.generate: prefix-match hit



Answer:
- Deep expertise: Apple is a company where experts lead



llama_print_timings:        load time =    8075.29 ms
llama_print_timings:      sample time =       4.63 ms /    16 runs   (    0.29 ms per token,  3452.74 tokens per second)
llama_print_timings: prompt eval time =   19623.04 ms /  2645 tokens (    7.42 ms per token,   134.79 tokens per second)
llama_print_timings:        eval time =    1004.21 ms /    15 runs   (   66.95 ms per token,    14.94 tokens per second)
llama_print_timings:       total time =   20752.21 ms


In [31]:
groundedness_rater_system_message = """
You are tasked with rating AI generated answers to questions posed by users.
You will be presented a question, context used by the AI system to generate the answer and an AI generated answer to the question.
In the input, the question will begin with ###Question, the context will begin with ###Context while the AI generated answer will begin with ###Answer.

Evaluation criteria:
The task is to judge the extent to which the metric is followed by the answer.
1 - The metric is not followed at all
2 - The metric is followed only to a limited extent
3 - The metric is followed to a good extent
4 - The metric is followed mostly
5 - The metric is followed completely

Metric:
The answer should be derived only from the information presented in the context

Instructions:
1. First write down the steps that are needed to evaluate the answer as per the metric.
2. Give a step-by-step explanation if the answer adheres to the metric considering the question and context as the input.
3. Next, evaluate the extent to which the metric is followed.
4. Use the previous information to rate the answer using the evaluaton criteria and assign a score.
"""

In [33]:
relevance_rater_system_message = """
You are tasked with rating AI generated answers to questions posed by users.
You will be presented a question, context used by the AI system to generate the answer and an AI generated answer to the question.
In the input, the question will begin with ###Question, the context will begin with ###Context while the AI generated answer will begin with ###Answer.

Evaluation criteria:
The task is to judge the extent to which the metric is followed by the answer.
1 - The metric is not followed at all
2 - The metric is followed only to a limited extent
3 - The metric is followed to a good extent
4 - The metric is followed mostly
5 - The metric is followed completely

Metric:
Relevance measures how well the answer addresses the main aspects of the question, based on the context.
Consider whether all and only the important aspects are contained in the answer when evaluating relevance.

Instructions:
1. First write down the steps that are needed to evaluate the context as per the metric.
2. Give a step-by-step explanation if the context adheres to the metric considering the question as the input.
3. Next, evaluate the extent to which the metric is followed.
4. Use the previous information to rate the context using the evaluaton criteria and assign a score.
"""

In [34]:
user_message_template = """
###Question
{question}

###Context
{context}

###Answer
{answer}
"""

In [35]:
user_input = "Who are the authors of this article and who published this article ?"


In [50]:
relavent_document=retriver.get_relevant_documents(user_input)
list_relavent_document=[d.page_content for d in  relavent_document]
context_for_query=".".join(list_relavent_document)
prompt=f"""[INST]{qna_system_message}\n
{'user'}:{qna_user_message_template.format(context=context_for_query,question=user_input)}[/INST]
"""

response= llm(prompt=prompt,
              temperature=0,
              top_p=0.95,
              max_tokens=1000,
              echo=False,
              stop=['INST'])
output_1= response["choices"][0]["text"]

Llama.generate: prefix-match hit

llama_print_timings:        load time =    8075.29 ms
llama_print_timings:      sample time =      16.48 ms /    40 runs   (    0.41 ms per token,  2427.48 tokens per second)
llama_print_timings: prompt eval time =   11138.12 ms /  1103 tokens (   10.10 ms per token,    99.03 tokens per second)
llama_print_timings:        eval time =    2234.83 ms /    39 runs   (   57.30 ms per token,    17.45 tokens per second)
llama_print_timings:       total time =   13631.99 ms


In [51]:
print(output_1)


The authors of the article "How Apple Is Organized for Innovation" are Joel M. Podolny and Morten T. Hansen. The article was published by Harvard Business Review.


In [48]:
groundness_prompt=f"""[INST]{groundedness_rater_system_message}\n
{'user'}:{user_message_template.format(question=user_input,context=context_for_query,answer=output)}[/INST]"""
response= llm(prompt=groundness_prompt,
              temperature=0.8,
              frequency_penalty=0,
              echo=False,
              max_tokens=1000,
              stop=['INST']
              )
output=response["choices"][0]["text"]

Llama.generate: prefix-match hit

llama_print_timings:        load time =    8075.29 ms
llama_print_timings:      sample time =      70.86 ms /   265 runs   (    0.27 ms per token,  3739.93 tokens per second)
llama_print_timings: prompt eval time =    2701.86 ms /    80 tokens (   33.77 ms per token,    29.61 tokens per second)
llama_print_timings:        eval time =   15769.98 ms /   264 runs   (   59.73 ms per token,    16.74 tokens per second)
llama_print_timings:       total time =   20012.71 ms


In [49]:
print(output)

 Based on the provided input, here are the steps to evaluate the answer:

1. Identify the question parts: Who are the authors of this article and who published this article?
2. Check if the information in the answer is present in the context: The names of the authors (Joel M. Podolny and Morten T. Hansen) and the publication details (Harvard Business Review, November–December 2020) are explicitly stated in the context.
3. Evaluate each question part individually to ensure that the answer is derived only from the context:
   a. For the first question part "Who are the authors of this article?", the context clearly states the names of the authors (Joel M. Podolny and Morten T. Hansen) multiple times in different parts of the text.
   b. For the second question part "Who published this article?", the context also mentions that the article is from Harvard Business Review, which confirms that the publication details are correct.
4. The answer is derived only from the information presented i

In [52]:
relevance_rater_prompt= f"""[INST]{relevance_rater_system_message}\n 
{'user'}:{user_message_template.format(question=user_input,context=context_for_query,answer=output_1)}[/INST]
"""
response= llm(prompt=relevance_rater_prompt,
              temperature=0,
              max_tokens=1000,
              top_k=40,
              top_p=0.95,
              echo=False,
              stop=['INST'])
output_2=response["choices"][0]['text']


Llama.generate: prefix-match hit

llama_print_timings:        load time =    8075.29 ms
llama_print_timings:      sample time =      75.57 ms /   189 runs   (    0.40 ms per token,  2500.99 tokens per second)
llama_print_timings: prompt eval time =   11026.22 ms /  1296 tokens (    8.51 ms per token,   117.54 tokens per second)
llama_print_timings:        eval time =   11430.18 ms /   188 runs   (   60.80 ms per token,    16.45 tokens per second)
llama_print_timings:       total time =   23446.05 ms


In [53]:
print(output_2)

Steps to evaluate context as per relevance metric:
1. Identify the main aspects of the question: Who are the authors? and Who published the article?
2. Determine if the context provides sufficient information for each aspect of the question.
3. Check if all important aspects are included in the context and only relevant information is provided.

The context adheres to the metric considering the question as follows:
1. The authors' names, Joel M. Podolny and Morten T. Hansen, are explicitly stated multiple times in the context.
2. The publication details, Harvard Business Review, are also mentioned several times.
3. No irrelevant information is present in the context.

The metric is followed mostly as all important aspects of the question have been addressed in the context and only relevant information has been provided.

Rating: 4 (Followed mostly)
